<a href="https://www.kaggle.com/code/ryancardwell/seawolfv6-1?scriptVersionId=271363762" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


# Seawolf v6 — Robust ARC-AGI Baseline (E2E)
Loads ARC-AGI JSONs (if present), applies robust solver, writes a valid `submission.json`.


In [1]:

# --- Cell 1: Imports & Paths ---
import os, json, time
from collections import Counter, defaultdict

DATA_DIRS = [
    "/kaggle/input/arc-prize-2025",
    "/kaggle/input/arc-agi",
    "/kaggle/working",
    "."
]
FILES = {
    "train_ch": "arc-agi_training_challenges.json",
    "train_sol": "arc-agi_training_solutions.json",
    "eval_ch": "arc-agi_evaluation_challenges.json",
    "eval_sol": "arc-agi_evaluation_solutions.json",
    "test_ch": "arc-agi_test_challenges.json",
    "sample": "sample_submission.json"
}

def find_file(name):
    for d in DATA_DIRS:
        p = os.path.join(d, name)
        if os.path.exists(p):
            return p
    return None

PATHS = {k: find_file(v) for k,v in FILES.items()}
print("[MANIFEST]", PATHS)


[MANIFEST] {'train_ch': '/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json', 'train_sol': '/kaggle/input/arc-prize-2025/arc-agi_training_solutions.json', 'eval_ch': '/kaggle/input/arc-prize-2025/arc-agi_evaluation_challenges.json', 'eval_sol': '/kaggle/input/arc-prize-2025/arc-agi_evaluation_solutions.json', 'test_ch': '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json', 'sample': '/kaggle/input/arc-prize-2025/sample_submission.json'}


In [2]:

# --- Cell 2: Load JSONs ---
def load_json(path, default=None):
    if path and os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    return default

TRAIN_CH = load_json(PATHS["train_ch"], {})
TRAIN_SOL = load_json(PATHS["train_sol"], {})
EVAL_CH  = load_json(PATHS["eval_ch"],  {})
EVAL_SOL = load_json(PATHS["eval_sol"], {})
TEST     = load_json(PATHS["test_ch"],  {})

print("[SIZES] train:", len(TRAIN_CH), "eval:", len(EVAL_CH), "test:", len(TEST))


[SIZES] train: 1000 eval: 120 test: 240


In [3]:

# --- Cell 3: Grid & Shape Utilities (hardened) ---
def grid_size(g):
    H = len(g) if g else 0
    W = len(g[0]) if H and g[0] else 0
    return H, W

def is_rect(g):
    if not g: return True
    W = len(g[0])
    return all(len(r) == W for r in g)

def pad_or_crop_to(g, H, W, fill=0):
    H = max(1, int(H)); W = max(1, int(W))
    h0, w0 = grid_size(g)
    gg = [row[:W] for row in (g[:H] if g else [])]
    while len(gg) < H: gg.append([fill]*W)
    for i in range(len(gg)):
        if len(gg[i]) < W: gg[i] = gg[i] + [fill]*(W - len(gg[i]))
    if not gg: gg = [[fill]*W]
    return gg

def repeat_to_shape(g, H, W):
    H = max(1, int(H)); W = max(1, int(W))
    h0, w0 = grid_size(g)
    if h0 == 0 or w0 == 0: g, h0, w0 = [[0]], 1, 1
    out = [[ g[r % h0][c % w0] for c in range(W) ] for r in range(H)]
    return out

def mode_color(g):
    if not g or not g[0]: return 0
    cnt = Counter(v for row in g for v in row)
    return cnt.most_common(1)[0][0]

def safe_base_patch(g, max_h=4, max_w=4):
    H, W = grid_size(g)
    if H == 0 or W == 0: return [[0]]
    h0 = max(1, min(H, max_h)); w0 = max(1, min(W, max_w))
    base = [row[:w0] for row in g[:h0]]
    if not base or not base[0]:
        base = [[mode_color(g)]]
    return base

def periodic_tile(g, Ht, Wt, period_hint=None):
    H, W = grid_size(g)
    if H == 0 or W == 0: return [[0]]
    if isinstance(period_hint, (tuple, list)) and len(period_hint) == 2:
        ph, pw = period_hint
        h0 = max(1, min(H, ph)); w0 = max(1, min(W, pw))
    elif isinstance(period_hint, int) and period_hint > 0:
        h0 = max(1, min(H, period_hint)); w0 = max(1, min(W, period_hint))
    else:
        h0 = max(1, min(H, 4)); w0 = max(1, min(W, 4))
    base = [row[:w0] for row in g[:h0]]
    if not base or not base[0]:
        base = safe_base_patch(g)
    return repeat_to_shape(base, Ht, Wt)

def force_shape(g, H, W=None, *, strategy="padcrop", fill=None):
    if isinstance(H, (tuple, list)) and W is None and len(H) == 2:
        H, W = H
    h0, w0 = grid_size(g)
    if W is None: W = w0 if w0 > 0 else H
    H = max(1, int(H)); W = max(1, int(W))
    fill_val = 0 if fill is None else fill
    if strategy == "repeat":
        base = safe_base_patch(g)
        return repeat_to_shape(base, H, W)
    return pad_or_crop_to(g if is_rect(g) else pad_or_crop_to(g, h0, w0, fill=fill_val), H, W, fill=fill_val)


In [4]:

# --- Cell 4: Target shape inference & recolor ---
def determine_target_shape(task):
    tr = task.get("train", [])
    out_shapes = {(len(ex["output"]), len(ex["output"][0])) for ex in tr if ex.get("output")}
    out_shapes = {(h,w) for (h,w) in out_shapes if h>0 and w>0}
    if len(out_shapes) == 1:
        return next(iter(out_shapes))

    scales = set(); ok = True
    for ex in tr:
        Ain, Aout = ex.get("input"), ex.get("output")
        if not Ain or not Aout: ok = False; break
        Hi, Wi = grid_size(Ain); Ho, Wo = grid_size(Aout)
        if Hi==0 or Wi==0: ok=False; break
        a, b = Ho/Hi, Wo/Wi
        if abs(round(a)-a)<1e-6 and abs(round(b)-b)<1e-6:
            scales.add((int(round(a)), int(round(b))))
        else:
            ok=False; break
    if ok and len(scales)==1:
        a,b = next(iter(scales))
        test_in = task["test"][0]["input"]
        Hi, Wi = grid_size(test_in)
        return max(1,a*Hi), max(1,b*Wi)

    test_in = task["test"][0]["input"]
    return grid_size(test_in)

def recolor_largest(g):
    return g


In [5]:

# --- Cell 5: Knobs ---
KNOBS = {
    "TILE_MAX_PERIOD": 4,
    "USE_REPEAT_WHEN_SHRINKING": True,
    "FALLBACK_FILL_MODE": "mode",
    "LOG_FIRST_N": 10,
}
print("[KNOBS]", KNOBS)


[KNOBS] {'TILE_MAX_PERIOD': 4, 'USE_REPEAT_WHEN_SHRINKING': True, 'FALLBACK_FILL_MODE': 'mode', 'LOG_FIRST_N': 10}


In [6]:

# --- Cell 6: Solver ---
def solve_one_input(Ain, target_hw):
    Ht, Wt = target_hw
    Hi, Wi = grid_size(Ain)
    use_repeat = KNOBS["USE_REPEAT_WHEN_SHRINKING"] and (Ht <= Hi and Wt <= Wi)
    if use_repeat:
        return periodic_tile(Ain, Ht, Wt, period_hint=KNOBS["TILE_MAX_PERIOD"])
    else:
        return force_shape(Ain, Ht, Wt, strategy="padcrop")

def solve_task(task):
    test_items = task.get("test", [])
    if not test_items:
        return [[[0]]]
    Ht, Wt = determine_target_shape(task)
    outs = []
    for item in test_items:
        Ain = item["input"]
        try:
            B = solve_one_input(Ain, (Ht, Wt))
            B = recolor_largest(B)
        except Exception as e:
            c = 0
            if KNOBS["FALLBACK_FILL_MODE"] == "mode":
                c = 0
                try:
                    c = Counter(v for row in Ain for v in row).most_common(1)[0][0]
                except Exception:
                    c = 0
            else:
                try:
                    c = int(KNOBS["FALLBACK_FILL_MODE"])
                except Exception:
                    c = 0
            B = [[c]*max(Wt,1) for _ in range(max(Ht,1))]
        outs.append(B)
    return outs


In [7]:

# --- Cell 7: Submission writer + validator ---
def validate_submission_dict(sub, expected_ids):
    problems = []
    missing = [tid for tid in expected_ids if tid not in sub]
    if missing:
        problems.append(f"missing ids: {missing[:10]}{'...' if len(missing)>10 else ''}")
    for tid in expected_ids:
        grids = sub.get(tid, None)
        if not isinstance(grids, list) or not grids:
            problems.append(f"{tid}: output must be non-empty list")
            continue
        for g in grids:
            H,W = len(g), len(g[0]) if g else 0
            if H<=0 or W<=0:
                problems.append(f"{tid}: grid must be HxW > 0, got {(H,W)}")
            for r in range(H):
                for c in range(W):
                    v = g[r][c]
                    if not isinstance(v, int) or not (0 <= v <= 9):
                        problems.append(f"{tid}: invalid cell value at {(r,c)} -> {v}")
                        break
    return problems

def write_submission_safe(sub, expected_ids, path="/kaggle/working/submission.json"):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    problems = validate_submission_dict(sub, expected_ids)
    if problems:
        print("[VALIDATOR] issues:", problems[:5])
    tmp = path + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(sub, f, ensure_ascii=False, separators=(",", ":"))
        f.flush(); os.fsync(f.fileno())
    os.replace(tmp, path)
    return path


In [8]:
# === FIX CELL — core deps, safe tiler, and clean adapter (run this once) ===
from typing import List, Tuple
import json, os, sys
from collections import Counter, deque

# ---------- Logging ----------
def log(msg, level="INFO"):
    print(f"[{level}] {msg}", file=sys.stderr)

def warn(tid, reason):
    log(f"{tid}: solver failed -> stub (reason: {reason})", level="WARN")

# ---------- Core grid helpers ----------
def grid_size(G: List[List[int]]) -> Tuple[int,int]:
    """Robustly infer (H,W) for list-of-lists grids; never returns (0,0)."""
    if not isinstance(G, list) or not G:
        return (1, 1)
    H = len(G)
    # allow ragged rows; choose min positive width among non-empty rows
    widths = [len(r) for r in G if isinstance(r, list)]
    W = max(1, min(widths) if widths else 1)
    return max(1, H), max(1, W)

def clamp_color(v: int) -> int:
    """Clamp to ARC palette 0..9."""
    try:
        x = int(v)
    except Exception:
        x = 0
    return 0 if x < 0 else 9 if x > 9 else x

def force_shape(G: List[List[int]], H: int, W: int, fill: int = 0) -> List[List[int]]:
    """Resize/pad/crop to (H,W) while clamping colors; never raises."""
    H = max(1, int(H)); W = max(1, int(W)); fill = clamp_color(fill)
    if not isinstance(G, list) or not G:
        return [[fill]*W for _ in range(H)]
    out = []
    for r in G:
        row = [clamp_color(x) for x in (r if isinstance(r, list) else [])]
        if len(row) < W: row = row + [fill]*(W - len(row))
        else:            row = row[:W]
        out.append(row)
    if len(out) < H: out += [[fill]*W for _ in range(H - len(out))]
    else:            out = out[:H]
    return out

# ---------- Safe periodic tiler (prevents IndexError on tiny bases) ----------
def periodic_tile(G: List[List[int]], Ht: int, Wt: int, max_period: int = 8) -> List[List[int]]:
    """Tile the top-left base patch over (Ht,Wt); guards against empty rows/period 0."""
    H, W = grid_size(G)
    Ht = max(1, int(Ht)); Wt = max(1, int(Wt))
    if H == 0 or W == 0:
        return [[0]*Wt for _ in range(Ht)]
    h0 = min(H, max(1, min(max_period, H)))
    w0 = min(W, max(1, min(max_period, W)))
    # Construct a non-empty base
    base = []
    for r in range(h0):
        row = G[r][:w0] if (isinstance(G[r], list) and G[r]) else [0]*w0
        if not row: row = [0]*w0
        base.append(row)
    # h0 and w0 are >= 1, base rows are non-empty, so modulo is safe
    return [[ base[r % h0][c % w0] for c in range(Wt) ] for r in range(Ht)]

# ---------- Clean ASCII adapter (replaces any non-breaking spaces) ----------
def solve_task_adapter(arg1, arg2=None, *, solver=None):
    """
    Accepts (task) or (tid, task). Passes to solver(task) and returns the output grid.
    """
    task = arg1 if arg2 is None else arg2
    if solver is None:
        raise ValueError("Provide solver=callable(task)->grid")
    return solver(task)

# ---------- Minimal safety stubs (used by your code) ----------
def mode_color(G: List[List[int]]) -> int:
    cnt = Counter([x for r in (G or []) for x in (r or []) if isinstance(x, int)])
    return max(cnt.items(), key=lambda kv: (kv[1], kv[0]))[0] if cnt else 0

def validate_and_repair_grid(G: List[List[int]]) -> List[List[int]]:
    H, W = grid_size(G)
    if H < 1 or W < 1:
        return [[0]]
    G2 = [[clamp_color(x) for x in (r if isinstance(r, list) else [0]*W)] for r in (G if isinstance(G, list) else [[0]*W])]
    return force_shape(G2, H, W, fill=0)

def safe_stub(task) -> List[List[int]]:
    test = task.get("test", [{"input":[[0]], "output":[[0]]}])
    test_in = test[0].get("input", [[0]])
    H, W = grid_size(test_in)
    m = mode_color(test_in)
    return [[m for _ in range(W)] for __ in range(H)]

# ---------- Optional: quick self-check (comment out if not needed) ----------
if __name__ == "__main__":
    # tiny sanity checks
    g = [[1,2],[3,4]]
    assert grid_size(g) == (2,2)
    assert force_shape([[9]], 2,3, fill=0) == [[9,0,0],[0,0,0]]
    t = periodic_tile([[1]], 2,2)
    assert t == [[1,1],[1,1]]
    log("FIX CELL loaded OK.")


[INFO] FIX CELL loaded OK.


In [9]:

# --- Cell 8: Runner (build submission) ---
start = time.time()
expected_ids = list(TEST.keys()) if isinstance(TEST, dict) else []
print(f"[RUNNER] TEST tasks: {len(expected_ids)}")
submission = {}
log_count = 0

for tid, task in (TEST.items() if isinstance(TEST, dict) else []):
    outs = solve_task(task)
    submission[tid] = outs
    if log_count < KNOBS["LOG_FIRST_N"]:
        shapes = [(len(g), len(g[0]) if g else 0) for g in outs]
        print(f"[LOG] {tid}: {len(outs)} test outs, shapes={shapes[:2]}{'...' if len(shapes)>2 else ''}")
        log_count += 1

out_path = write_submission_safe(submission, expected_ids, "/kaggle/working/submission.json")
size = os.path.getsize(out_path) if os.path.exists(out_path) else 0
print("[SUBMISSION] wrote", out_path, "size:", size, "bytes")
print("[TIME] sec:", round(time.time()-start, 2))


[RUNNER] TEST tasks: 240
[LOG] 00576224: 1 test outs, shapes=[(6, 6)]
[LOG] 007bbfb7: 1 test outs, shapes=[(9, 9)]
[LOG] 009d5c81: 1 test outs, shapes=[(14, 14)]
[LOG] 00d62c1b: 1 test outs, shapes=[(20, 20)]
[LOG] 00dbd492: 1 test outs, shapes=[(20, 20)]
[LOG] 017c7c7b: 1 test outs, shapes=[(9, 3)]
[LOG] 025d127b: 1 test outs, shapes=[(10, 10)]
[LOG] 03560426: 1 test outs, shapes=[(10, 10)]
[LOG] 045e512c: 1 test outs, shapes=[(21, 21)]
[LOG] 0520fde7: 1 test outs, shapes=[(3, 3)]
[SUBMISSION] wrote /kaggle/working/submission.json size: 124063 bytes
[TIME] sec: 0.15


In [10]:

# --- Cell 9: Inspect few entries & summary ---
from collections import defaultdict
sizes = defaultdict(int)
for tid, outs in submission.items():
    for g in outs:
        sizes[(len(g), len(g[0]) if g else 0)] += 1

print("[SUMMARY] top shapes:", sorted(sizes.items(), key=lambda x: -x[1])[:10])
if submission:
    sample_tid = next(iter(submission))
    sg = submission[sample_tid][0]
    print("[SAMPLE]", sample_tid, "-> first grid HxW:", (len(sg), len(sg[0]) if sg else 0))
    for row in sg[:min(5,len(sg))]:
        print(row[:min(10, len(row))])


[SUMMARY] top shapes: [((10, 10), 29), ((16, 16), 15), ((3, 3), 12), ((9, 9), 11), ((13, 13), 10), ((15, 15), 8), ((30, 30), 7), ((5, 5), 7), ((1, 1), 6), ((14, 14), 5)]
[SAMPLE] 00576224 -> first grid HxW: (6, 6)
[3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3]
